#Creating the Dataset

In [2]:
!pip install requests

In [3]:
token = ""

In [8]:
import os, time, requests, pandas as pd
from pathlib import Path

def fetch_all_issues(owner="scikit-learn", repo="scikit-learn", token=None, issues_path=Path(".")):
    headers = {"Authorization": f"token {token}"}
    url = f"https://api.github.com/repos/{owner}/{repo}/issues"
    params = {"state": "all", "per_page": 100, "page": 1}
    all_issues = []

    while params["page"] < 99:
        resp = requests.get(url, headers=headers, params=params)
        if resp.status_code == 403:
            reset = int(resp.headers.get("X-RateLimit-Reset", time.time() + 60))
            wait = max(reset - time.time(), 60)
            time.sleep(wait + 1)
            continue

        resp.raise_for_status()
        page_data = resp.json()
        if not page_data:
            break

        all_issues.extend(page_data)
        print(f"Fetched page {params['page']}, total so far {len(all_issues)}")

        if 'rel="next"' not in resp.headers.get("Link", ""):
            break

        params["page"] += 1

    df = pd.DataFrame.from_records(all_issues)
    out = issues_path / f"{repo}-issues.jsonl"
    df.to_json(out, orient="records", lines=True)
    print(f"Downloaded {len(all_issues)} issues -> {out}")

In [9]:
fetch_all_issues(token=token)

Fetched page 1, total so far 100
Fetched page 2, total so far 200
Fetched page 3, total so far 300
Fetched page 4, total so far 400
Fetched page 5, total so far 500
Fetched page 6, total so far 600
Fetched page 7, total so far 700
Fetched page 8, total so far 800
Fetched page 9, total so far 900
Fetched page 10, total so far 1000
Fetched page 11, total so far 1100
Fetched page 12, total so far 1200
Fetched page 13, total so far 1300
Fetched page 14, total so far 1400
Fetched page 15, total so far 1500
Fetched page 16, total so far 1600
Fetched page 17, total so far 1700
Fetched page 18, total so far 1800
Fetched page 19, total so far 1900
Fetched page 20, total so far 2000
Fetched page 21, total so far 2100
Fetched page 22, total so far 2200
Fetched page 23, total so far 2300
Fetched page 24, total so far 2400
Fetched page 25, total so far 2500
Fetched page 26, total so far 2600
Fetched page 27, total so far 2700
Fetched page 28, total so far 2800
Fetched page 29, total so far 2900
Fet

In [10]:
!cp scikit-learn-issues.jsonl /content/drive/MyDrive/Datasets/Github_Issues/

In [11]:
import json

with open("/content/drive/MyDrive/Datasets/Github_Issues/scikit-learn-issues.jsonl") as f:
    for i, line in enumerate(f):
        if i == 1:
            break
        print(json.loads(line))

{'url': 'https://api.github.com/repos/scikit-learn/scikit-learn/issues/32421', 'repository_url': 'https://api.github.com/repos/scikit-learn/scikit-learn', 'labels_url': 'https://api.github.com/repos/scikit-learn/scikit-learn/issues/32421/labels{/name}', 'comments_url': 'https://api.github.com/repos/scikit-learn/scikit-learn/issues/32421/comments', 'events_url': 'https://api.github.com/repos/scikit-learn/scikit-learn/issues/32421/events', 'html_url': 'https://github.com/scikit-learn/scikit-learn/pull/32421', 'id': 3491965948, 'node_id': 'PR_kwDOAAzd1s6shTdZ', 'number': 32421, 'title': 'MNT Switch to absolute imports in sklearn/utils/_fast_dict.pxd', 'user': {'login': 'dipak0000812', 'id': 217148765, 'node_id': 'U_kgDODPFtXQ', 'avatar_url': 'https://avatars.githubusercontent.com/u/217148765?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/dipak0000812', 'html_url': 'https://github.com/dipak0000812', 'followers_url': 'https://api.github.com/users/dipak0000812/followers', 'fol

In [12]:
import json

with open("/content/drive/MyDrive/Datasets/Github_Issues/scikit-learn-issues.jsonl") as f:
    first_line = f.readline()
    first_entry = json.loads(first_line)
    print(first_entry.keys())

dict_keys(['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'type', 'active_lock_reason', 'draft', 'pull_request', 'body', 'closed_by', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason', 'sub_issues_summary', 'issue_dependencies_summary'])


In [22]:
import json

fields_to_keep = [
    'id',"title", "state", "comments", "labels",
    "created_at", "updated_at", "closed_at", "body", "user", "pull_request"
]

def simplify_issue(issue):
    simplified = {k: issue.get(k) for k in fields_to_keep}
    if "user" in simplified and isinstance(simplified["user"], dict):
        simplified["user"] = simplified["user"].get("login")
    return simplified

with open("/content/drive/MyDrive/Datasets/Github_Issues/scikit-learn-issues.jsonl", "r") as infile, open("scikit-learn-issues-slim.jsonl", "w") as outfile:
    for line in infile:
        issue = json.loads(line)
        clean = simplify_issue(issue)
        outfile.write(json.dumps(clean) + "\n")

In [45]:
from datasets import load_dataset

scikit_issues_dataset = load_dataset("json", data_files="scikit-learn-issues-slim.jsonl", split="train")
print(scikit_issues_dataset)

Dataset({
    features: ['id', 'title', 'state', 'comments', 'labels', 'created_at', 'updated_at', 'closed_at', 'body', 'user', 'pull_request'],
    num_rows: 9800
})


In [46]:
def filter_pull_requests(record):
  return record["pull_request"] is not None

scikit_issues_dataset = scikit_issues_dataset.filter(filter_pull_requests)

In [47]:
print(scikit_issues_dataset)

Dataset({
    features: ['id', 'title', 'state', 'comments', 'labels', 'created_at', 'updated_at', 'closed_at', 'body', 'user', 'pull_request'],
    num_rows: 7033
})


In [48]:
def map_labels(records):
  labels_list = []
  for labels in records['labels']:
    labels_list.append([l['name'] for l in labels])
  records['labels_list'] = labels_list
  del records['labels']
  return records

In [49]:
scikit_issues_dataset = scikit_issues_dataset.map(map_labels, batched=True)

Map:   0%|          | 0/7033 [00:00<?, ? examples/s]

In [50]:
print(scikit_issues_dataset[0]['labels_list'])

['module:utils', 'cython']


In [51]:
print(scikit_issues_dataset)

Dataset({
    features: ['id', 'title', 'state', 'comments', 'created_at', 'updated_at', 'closed_at', 'body', 'user', 'pull_request', 'labels_list'],
    num_rows: 7033
})


In [54]:
def filter_data(records):
  labels_lists = records['labels_list']
  return [len(labels) > 0 for labels in labels_lists]

scikit_issues_dataset = scikit_issues_dataset.filter(filter_data, batched=True)

Filter:   0%|          | 0/7033 [00:00<?, ? examples/s]

In [55]:
print(scikit_issues_dataset)

Dataset({
    features: ['id', 'title', 'state', 'comments', 'created_at', 'updated_at', 'closed_at', 'body', 'user', 'pull_request', 'labels_list'],
    num_rows: 6402
})


In [58]:
def map_text(records):
  titles = records['title']
  bodies = records['body']
  text_list = [title + "[SEP]" + (body if body is not None else "") for title, body in zip(titles, bodies)]
  return {'text': text_list}

scikit_issues_dataset = scikit_issues_dataset.map(map_text, batched=True)

Map:   0%|          | 0/6402 [00:00<?, ? examples/s]

In [59]:
print(scikit_issues_dataset)

Dataset({
    features: ['id', 'title', 'state', 'comments', 'created_at', 'updated_at', 'closed_at', 'body', 'user', 'pull_request', 'labels_list', 'text'],
    num_rows: 6402
})


In [60]:
scikit_issues_dataset = scikit_issues_dataset.remove_columns(['title', 'state', 'comments', 'created_at', 'updated_at', 'closed_at', 'body', 'user', 'pull_request'])
print(scikit_issues_dataset)

Dataset({
    features: ['id', 'labels_list', 'text'],
    num_rows: 6402
})


In [63]:
unique_label_count = {}
for record in scikit_issues_dataset:
  for label in record['labels_list']:
    if label in unique_label_count:
      unique_label_count[label] += 1
    else:
      unique_label_count[label] = 1

print(unique_label_count)

{'module:utils': 793, 'cython': 521, 'module:metrics': 730, 'Build / CI': 806, 'Documentation': 1998, 'No Changelog Needed': 1798, 'module:tree': 149, 'module:cluster': 300, 'module:datasets': 257, 'Quick Review': 939, 'module:svm': 67, 'module:manifold': 88, 'module:decomposition': 167, 'module:linear_model': 433, 'module:preprocessing': 226, 'Closing candidate': 6, 'module:neighbors': 152, 'module:feature_extraction': 79, 'Waiting for Second Reviewer': 321, 'Array API': 132, 'module:impute': 58, 'module:ensemble': 306, 'module:feature_selection': 117, 'dependencies': 19, 'module:cross_decomposition': 27, 'spam': 50, 'Performance': 55, 'module:model_selection': 182, 'Waiting for Reviewer': 124, 'module:calibration': 6, 'module:neural_network': 38, 'module:compose': 63, 'Metadata Routing': 14, 'module:pipeline': 59, 'frontend': 7, 'module:covariance': 72, 'API': 13, 'module:inspection': 68, 'Needs Decision': 21, 'module:test-suite': 30, 'module:semi_supervised': 18, 'module:mixture': 3

In [68]:
MIN_LABEL_COUNT = 50
labels_to_delete = set()
for k,v in unique_label_count.items():
  if v<50:
    labels_to_delete.add(k)
print(labels_to_delete)

{'Developer API', 'float32', 'Refactor', 'RFC', 'Superseded', 'module:naive_bayes', 'hacktoberfest-accepted', 'Needs Decision', 'help wanted', 'Needs Benchmarks', 'module:random_projection', 'module:mixture', 'C/C++', 'module:cross_decomposition', 'Metadata Routing', 'Numerical Stability', 'free-threading', 'module:multioutput', 'module:calibration', 'High Priority', 'module:semi_supervised', 'Blocker', 'Regression', 'Needs Reproducible Code', 'Moderate', 'module:test-suite', 'Needs work', 'API', 'Pandas compatibility', 'module:kernel_approximation', 'Enhancement', 'Needs Decision - Include Feature', 'module:gaussian_process', 'workflow', 'Breaking Change', 'module:neural_network', 'module:discriminant_analysis', 'OS:Windows', 'module:multiclass', 'frontend', 'Closing candidate', 'good first issue', 'good first PR to review', 'CUDA CI', 'Needs Decision - Close', 'dependencies', 'New Feature', 'Packaging', 'Bug'}


In [69]:
def map_min_labels(records):
  labels_lists = records['labels_list']
  filtered_labels_lists = [[label for label in labels if label not in labels_to_delete] for labels in labels_lists]
  records['labels_list'] = filtered_labels_lists
  return records

scikit_issues_dataset = scikit_issues_dataset.map(map_min_labels, batched=True)

Map:   0%|          | 0/6402 [00:00<?, ? examples/s]

In [71]:
print(len(unique_label_count))
unique_label_count = {}
for record in scikit_issues_dataset:
  for label in record['labels_list']:
    if label in unique_label_count:
      unique_label_count[label] += 1
    else:
      unique_label_count[label] = 1
print(len(unique_label_count))

82
33


In [72]:
label_to_id={}
id_to_label={}
id = 0
for key in unique_label_count:
  label_to_id[key] = id
  id_to_label[id] = key
  id += 1

In [73]:
import numpy as np
def encode(records):
  labels_lists = records['labels_list']
  batch_multi_hot_vectors = []
  for issue_labels in labels_lists:
    multi_hot_vector = np.zeros(len(label_to_id), dtype=float)
    for label in issue_labels:
      if label in label_to_id:
        multi_hot_vector[label_to_id[label]] = 1.0
    batch_multi_hot_vectors.append(multi_hot_vector.tolist())
  return {'labels' : batch_multi_hot_vectors}

scikit_issues_dataset_vectored = scikit_issues_dataset.map(encode, batched = True)

Map:   0%|          | 0/6402 [00:00<?, ? examples/s]

In [76]:
scikit_issues_dataset_vectored = scikit_issues_dataset_vectored.remove_columns('labels_list')
print(scikit_issues_dataset_vectored)

Dataset({
    features: ['id', 'text', 'labels'],
    num_rows: 6402
})


In [79]:
!pip install scikit-multilearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 3.0 MB/s eta 0:00:00


In [81]:
from skmultilearn.model_selection import iterative_train_test_split
from datasets import Dataset, DatasetDict, load_dataset
def stratified_split(dataset, N):
    """Performs the two-step stratified 70/15/15 split."""

    X_ids = np.array(dataset['id']).reshape(-1, 1)
    X_text = np.array(dataset['text']).reshape(-1, 1)

    X = np.hstack((X_ids, X_text))
    Y = np.array(dataset['labels'])

    X_temp, Y_temp, X_test, Y_test = iterative_train_test_split(X, Y, test_size=0.15)

    # Separate Train (70%) and Validation (15%)
    val_size_ratio = 0.15 / 0.85
    X_train, Y_train, X_val, Y_val = iterative_train_test_split(X_temp, Y_temp, test_size=val_size_ratio)

    return DatasetDict({
        'train': Dataset.from_dict({"id": X_train[:, 0].flatten().tolist(),"text": X_train[:, 1].flatten().tolist(), "labels": Y_train.tolist()}),
        'validation': Dataset.from_dict({"id": X_val[:, 0].flatten().tolist(),"text": X_val[:, 1].flatten().tolist(), "labels": Y_val.tolist()}),
        'test': Dataset.from_dict({"id": X_test[:, 0].flatten().tolist(),"text": X_test[:, 1].flatten().tolist(), "labels": Y_test.tolist()})
    })

stratified_datasets = stratified_split(scikit_issues_dataset_vectored, len(label_to_id))
print(stratified_datasets)

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'labels'],
        num_rows: 4481
    })
    validation: Dataset({
        features: ['id', 'text', 'labels'],
        num_rows: 960
    })
    test: Dataset({
        features: ['id', 'text', 'labels'],
        num_rows: 961
    })
})


In [82]:
from huggingface_hub import notebook_login

notebook_login()

In [84]:
stratified_datasets.push_to_hub("scikit-learn-issues")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

...

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              : 100%|##########|  368kB /  368kB            

CommitInfo(commit_url='https://huggingface.co/datasets/devangb4/scikit-learn-issues/commit/664a4eeced1de409723f57ccf4436ef5d7eea1b6', commit_message='Upload dataset', commit_description='', oid='664a4eeced1de409723f57ccf4436ef5d7eea1b6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/devangb4/scikit-learn-issues', endpoint='https://huggingface.co', repo_type='dataset', repo_id='devangb4/scikit-learn-issues'), pr_revision=None, pr_num=None)

In [92]:
from huggingface_hub import HfApi

api = HfApi()
repo_id = "devangb4/scikit-learn-issues"

api.upload_file(
    path_or_fileobj="README.md",
    path_in_repo="README.md",
    repo_id=repo_id,
    repo_type="dataset",
)


README.md uploaded to devangb4/scikit-learn-issues on the Hugging Face Hub.


In [93]:
import json

mappings = {
    "label_to_id": label_to_id,
    "id_to_label": id_to_label
}

with open("label_mappings.json", "w") as f:
    json.dump(mappings, f)

Label mappings saved to label_mappings.json


In [94]:
from huggingface_hub import HfApi

api = HfApi()

repo_id = "devangb4/scikit-learn-issues"

api.upload_file(
    path_or_fileobj="label_mappings.json",
    path_in_repo="label_mappings.json",
    repo_id=repo_id,
    repo_type="dataset",
)


Label mappings file uploaded to devangb4/scikit-learn-issues on the Hugging Face Hub.


#Training the Model

In [1]:
checkpoint = "distilbert-base-uncased"
new_checkpoint = "FacebookAI/roberta-base"
deberta_checkpoint = "microsoft/deberta-v3-base"

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(deberta_checkpoint)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [4]:
from datasets import load_dataset
dataset = load_dataset("devangb4/scikit-learn-issues")
print(dataset)

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.52M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/381k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/368k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4481 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/960 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/961 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'labels'],
        num_rows: 4481
    })
    validation: Dataset({
        features: ['id', 'text', 'labels'],
        num_rows: 960
    })
    test: Dataset({
        features: ['id', 'text', 'labels'],
        num_rows: 961
    })
})


In [5]:
def tokenize_function(example):
  return tokenizer(example['text'], truncation = True)
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['text', 'id'])
print(tokenized_dataset)

Map:   0%|          | 0/4481 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/960 [00:00<?, ? examples/s]

Map:   0%|          | 0/961 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4481
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 960
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 961
    })
})


In [6]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [7]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./deberta_multilabel_clean_results",
    num_train_epochs=8,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    logging_steps=100
)

In [8]:
import torch
from sklearn.metrics import f1_score, accuracy_score, hamming_loss
def compute_multilabel_metrics(eval_pred):
    """Calculates all required multi-label metrics based on a 0.5 threshold."""
    logits, labels = eval_pred

    probabilities = torch.sigmoid(torch.from_numpy(logits))

    predictions = (probabilities > 0.5).int().numpy()

    return {
        "f1_micro": f1_score(labels, predictions, average="micro", zero_division=0),
        "f1_macro": f1_score(labels, predictions, average="macro", zero_division=0),
        "subset_accuracy": accuracy_score(labels, predictions),
        "hamming_loss": hamming_loss(labels, predictions),
    }

In [9]:
from huggingface_hub import hf_hub_download
import json

repo_id = "devangb4/scikit-learn-issues"
file_path = "label_mappings.json"

downloaded_file_path = hf_hub_download(repo_id=repo_id, filename=file_path, repo_type="dataset")

with open(downloaded_file_path, "r") as f:
    mappings = json.load(f)

label_to_id = mappings["label_to_id"]
id_to_label = {int(k): v for k, v in mappings["id_to_label"].items()} # Convert keys to int

print("Label mappings loaded successfully.")
print(f"label_to_id: {label_to_id}")
print(f"id_to_label: {id_to_label}")

label_mappings.json: 0.00B [00:00, ?B/s]

Label mappings loaded successfully.
label_to_id: {'module:utils': 0, 'cython': 1, 'module:metrics': 2, 'Build / CI': 3, 'Documentation': 4, 'No Changelog Needed': 5, 'module:tree': 6, 'module:cluster': 7, 'module:datasets': 8, 'Quick Review': 9, 'module:svm': 10, 'module:manifold': 11, 'module:decomposition': 12, 'module:linear_model': 13, 'module:preprocessing': 14, 'module:neighbors': 15, 'module:feature_extraction': 16, 'Waiting for Second Reviewer': 17, 'Array API': 18, 'module:impute': 19, 'module:ensemble': 20, 'module:feature_selection': 21, 'spam': 22, 'Performance': 23, 'module:model_selection': 24, 'Waiting for Reviewer': 25, 'module:compose': 26, 'module:pipeline': 27, 'module:covariance': 28, 'module:inspection': 29, 'To backport': 30, 'Stalled': 31, 'Validation': 32}
id_to_label: {0: 'module:utils', 1: 'cython', 2: 'module:metrics', 3: 'Build / CI', 4: 'Documentation', 5: 'No Changelog Needed', 6: 'module:tree', 7: 'module:cluster', 8: 'module:datasets', 9: 'Quick Review',

In [10]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
    deberta_checkpoint,
    num_labels = len(label_to_id),
    id2label = id_to_label,
    label2id = label_to_id,
    problem_type="multi_label_classification",
)

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import Trainer
trainer = Trainer(
    model = model,
    args=training_args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['validation'],
    data_collator = data_collator,
    tokenizer=tokenizer,
    compute_metrics = compute_multilabel_metrics
)
trainer.train()

/tmp/ipython-input-2022721879.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: devangborkar3 (devangborkar3-uc-davis) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 19.46 GiB. GPU 0 has a total capacity of 79.32 GiB of which 7.58 GiB is free. Process 35107 has 71.73 GiB memory in use. Of the allocated memory 67.57 GiB is allocated by PyTorch, and 3.66 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [30]:
test_results = trainer.evaluate(tokenized_dataset["test"])
print(json.dumps(test_results, indent=4))

{
    "eval_loss": 0.11660105735063553,
    "eval_f1_micro": 0.6342329545454546,
    "eval_f1_macro": 0.21041294223927934,
    "eval_subset_accuracy": 0.32362122788761705,
    "eval_hamming_loss": 0.03247879418534986,
    "eval_runtime": 28.7435,
    "eval_samples_per_second": 33.434,
    "eval_steps_per_second": 1.079,
    "epoch": 8.0
}


In [31]:

model.push_to_hub("scikit-issues-multilabel-classification", commit_description="Retrained the model on clean data after removing id field")
tokenizer.push_to_hub("scikit-issues-multilabel-classification")

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...gewofu1/model.safetensors:   0%|          | 16.3kB /  499MB            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/devangb4/scikit-issues-multilabel-classification/commit/5c31ba3b4eff6d0bfff65a922cc2b82c3cd4f396', commit_message='Upload tokenizer', commit_description='', oid='5c31ba3b4eff6d0bfff65a922cc2b82c3cd4f396', pr_url=None, repo_url=RepoUrl('https://huggingface.co/devangb4/scikit-issues-multilabel-classification', endpoint='https://huggingface.co', repo_type='model', repo_id='devangb4/scikit-issues-multilabel-classification'), pr_revision=None, pr_num=None)